# Catalogue pre-processing

In [ ]:
# cleaner avoid to restart kernel for each code modification, use it just when alone
from cleaner import modules_cleaner ; modules_cleaner()
# load current project if secondary ipynb runned alone
import metys; metys.Metys.secondary()

In [ ]:
%matplotlib inline
import re
import os
import numpy
import subprocess
import cPickle as pickle

from copy import deepcopy

import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
from mpl_toolkits.basemap import Basemap

from hmtk.seismicity.occurrence.weichert import Weichert
from hmtk.plotting.seismicity.occurrence.recurrence_plot import plot_trunc_gr_model

from hmtk.seismicity.completeness.comp_stepp_1971 import Stepp1971 
from hmtk.plotting.seismicity.completeness.plot_stepp_1972 import create_stepp_plot
from hmtk.plotting.seismicity.catalogue_plots import (plot_magnitude_time_scatter, 
                                                      plot_magnitude_time_density)

from hmtk.seismicity.selector import CatalogueSelector
from hmtk.seismicity.declusterer.dec_gardner_knopoff import GardnerKnopoffType1
from hmtk.seismicity.declusterer.distance_time_windows import GardnerKnopoffWindow

## Read current model and fix configuration parameters

In [ ]:
model_key = metys.g_prj.models.current 
cutoff_magnitude = float(metys.g_prj.mod[model_key]['catalogue_cutoff_magnitude'])
cutoff_depth = float(metys.g_prj.mod[model_key]['catalogue_cutoff_depth'])
binwidth = float(metys.g_prj.mod[model_key]['mfd_binwidth'])
print 'MFD bin width    : %.2f' % (binwidth)
print 'Magnitude cutoff : %.2f' % (cutoff_magnitude)

## Read the catalogue

In [ ]:
catalogue = metys.g_prj.mod['Model01']['earthquake_catalogue']

In [ ]:
selector = CatalogueSelector(catalogue, create_copy=False)
selector.within_magnitude_range(cutoff_magnitude, 10.)
selector.within_depth_range(300, 0)
#
print 'Catalogue minimum depth [km]: %.2f ' % numpy.min(catalogue.data['depth'])
print 'Catalogue maximum depth [km]: %.2f ' % numpy.max(catalogue.data['depth'])

## Declustering

In [ ]:
distance_time_wind = GardnerKnopoffWindow()
config = {'time_distance_window': distance_time_wind, 'fs_time_prop': .9}


declusterer = GardnerKnopoffType1()
vcl, flag = declusterer.decluster(catalogue, config)
catalogue_original = deepcopy(catalogue)
print 'Original number of events', catalogue_original.get_number_events()
catalogue.select_catalogue_events(numpy.where(flag == 0)[0])
print 'Number of mainshocks:', len(catalogue.data['magnitude'])

## Plotting

In [ ]:
# Computing geograohic limits of the catalogue
lomin = min(catalogue.data['longitude'])
lomax = max(catalogue.data['longitude'])
lamin = min(catalogue.data['latitude'])
lamax = max(catalogue.data['latitude'])    
lon_0 = sum([lomin,lomax])/2
lat_0 = sum([lamin,lamax])/2

if 1: 
    fig = plt.figure(figsize=(20,16))
    m = Basemap(llcrnrlon=lomin,
                llcrnrlat=lamin,
                urcrnrlon=lomax,
                urcrnrlat=lamax,
                resolution='i',
                projection='tmerc',
                lon_0=lon_0,
                lat_0=lat_0)

    m.shadedrelief()
    
    parallels = numpy.arange(lamin, lamax, 5.)
    m.drawparallels(parallels,labels=[False,True,True,False], fontsize=16)
    meridians = numpy.arange(lomin, lomax, 5.)
    m.drawmeridians(meridians,labels=[True,False,False,True], fontsize=16)
    
    # Coordinate conversion
    idx = numpy.nonzero(flag!=0)[0]
    x, y = m(catalogue.data['longitude'], catalogue.data['latitude'])
    
    print idx.shape
    print x.shape

    # catalogue    
    x, y = m(catalogue.data['longitude'], catalogue.data['latitude'])
    plt.plot(x, y, 'o', linewidth=6, alpha=0.4, color='blue')
    #plt.scatter(x, y, catalogue.data['magnitude'], marker='o', linewidth=0.5, alpha=0.4, edgecolors='none')
    
    x, y = m(catalogue_original.data['longitude'], catalogue_original.data['latitude'])
    plt.plot(x[idx], y[idx], 'x', linewidth=.5, alpha=0.8, color='red') 
    
    

## Completeness analysis

In [ ]:
# Declustering configuration parameters
config = {'magnitude_bin': .5, 'time_bin': 10., 'increment_lock': True}
stepp = Stepp1971()
compl_table = stepp.completeness(catalogue, config)          
subprocess.call('rm aa', shell=True)
# Info
print 'Maximum magnitude is: ', max(catalogue.data['magnitude'])
print 'The catalogue contains %d events' % (catalogue.get_number_events())
print 'Completeness table: \n'
# Display html
for line in compl_table:
    print '%.2f, %.2f' % (line[0], line[1])
# Plot completeness  
ppp = plot_magnitude_time_density(catalogue, mag_int=0.1, time_int=10, normalisation=True, 
                                  bootstrap=0, completeness=compl_table)

## GR Relationship

In [ ]:
idx = numpy.nonzero(compl_table[:,1] < numpy.max(catalogue.data['magnitude']))
weichert_config = {'magnitude_interval': 0.1, 
                   'reference_magnitude': 0.0}
weichert = Weichert()
bval_wei, sigmab, aval_wei, sigmaa = weichert.calculate(catalogue, weichert_config, completeness=compl_table)
print 'bval: %.3f (sigma=%.3f)' % (bval_wei, sigmab)
print 'aval: %.3f (sigma=%.3f)' % (aval_wei, sigmaa)

In [ ]:
plot_trunc_gr_model(aval_wei, bval_wei, 5.0, 8.3, 0.1, catalogue=catalogue,
        completeness=compl_table)

## Saving the catalogue

In [ ]:
output_path = os.path.join(metys.g_prj.folder, '%s_catalogue_declustered.pkl' % model_key)
fou = open(output_path,'wb') 
pickle.dump(catalogue, fou)
fou.close()
print 'Catalogue dataset saved into file: \n%s' % (output_path)